In [2]:
import os
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [61]:
# get image from directory and normalize image
test_data = 'datatest'
train_data = 'datatrain'

train_data_generator = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    )
test_data_generator = ImageDataGenerator(rescale=1./255)

train_generator = train_data_generator.flow_from_directory(
    train_data,
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
    
)
test_generator = test_data_generator.flow_from_directory(
    test_data,
    target_size=(224, 224),
    batch_size=128,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)
print(f"Number of classes: {train_generator.num_classes}")
print(f"Class indices: {train_generator.class_indices}")
print(f"Number of samples: {train_generator.samples}")
print(f"Number of samples: {test_generator.samples}")


Found 12000 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Number of classes: 3
Class indices: {'Boot': 0, 'Sandal': 1, 'Shoe': 2}
Number of samples: 12000
Number of samples: 150


In [62]:
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")
print("Num GPUs Available: ", tf.config.list_physical_devices())


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1
Num GPUs Available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [63]:
# Create Model of CNN
with strategy.scope():
  model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')

  ])
  model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=test_generator)
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc}')